# GRO620 - Activité procédurale 1

Dans cette activité, nous allons principalement travailler sur les éléments nécessaires pour capter une image numériquement, les transformations entre repères 2D et 3D, et l'encodage numérique de la couleur.

In [1]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## Acquisition et caractéristiques de la lumière

### Q1.1

À partir de la figure 2.23 du livre de référence, décrivez en une phrase le rôle de chacune des étapes de la chaîne d'acquisition d'images numériques.

Optics : Redirection de la lumière pour que l'image soit dans le bon sens et de la bonne grandeur.
Réponse : Apporter la lumière au capteur.

Aperture : Gestion de la grandeur du trou qui laisse passer la lumière, de la quanité et de l'intensité de la lumière qui se rend aux capteurs
Réponse : Meilleure profondeur de champs, pour flouter ce qu'on désir

Shutter : Gestion du temps d'exposition du capteur à la lumière pour modifier le depth of field ou le motion blur dans le cas d'image dynamique
Réponse : si c'est sombre, on peut s'exposer plus longtemps pour augmenter la luminosité, peut augmenter le motion blur

Sensor : Récolte des photons et accumulation de ceux-ci
Réponse : Transformation des photons en signaux électrique

Gain : Amplification des signaux électrique provoqué par les photon
Réponse : Peut faire en sorte d'amplifier le bruit

ADC : Conversion des signaux analogique en signaux numérique

Mosaic : C'est le passage d'un "color filter array" aux valeurs RGB
Réponse : Le CFA c'est des pattern comme le Bayer pattern, on cherche à faire la moyenne des couleurs autour de celle percu pour donner une valeur au au couleur qui ne sont pas mesuré

Denoise and sharpen : On enlève le bruit dans l'image (Avec du digital signal processing)
Réponse : On enlève le bruit et défini mieux les contours des informations provenant de la dernière étape

White balance : Approché les point blanc de l'iamge d'un blanc pure pour compenser les grande illuminations colorées
Réponse :

Gamma : mapping of the luminance values through a gamma function to increase the perceived dynamic range of the signal.
Réponse :On compense avec la fonction comment les yeux voient vraiment les choses, ce qu'on peut percevoir va apparaître plus 

Compress : réduction de la taille du fichier pour que ce soit utilisable.
Réponse : Il peut y avoir des pertes de résolution de la couleur

### Q1.2

Quelle est la différence entre paramètres intrinsèques et extrinsèques du caméra ? Décrivez chaque type en une phrase.

Paramètre propres à la caméra : point focal, aperture max, etc. = Intrinséque
Paramètre provenant de l'environnement : Illumination, mouvement,ou est la caméra etc. = extrinsèque, 
 Différence : L'un représente les paramètre propre à la caméra et qui ne change pas tandis que l'autre représente des paramètres propres à l'environnement, qui eux peuvent changer.

### Q1.3

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

Le capteur de cette caméra a une taille de 30 mm x 15 mm.

Pouvez-vous estimer la distance focale en mm de la lentille de cette caméra à partir de la matrice $K$ ?

In [44]:
# Réponse ici.
import numpy as np
w = 30
K = np.array([[620.,   0., 1024.],
              [  0., 620.,  512.],
              [  0.,   0.,    1.]
])

f = K[0,0]*(30/(1024*2))
print(f)

#La matrice K est en pixel, on veut savoir le rapport mm/pixel et on
# on multiplie avec la distance focal en mm
#1024 est choisi pour C_x car cela représente le centre de la caméra


9.08203125


### Q1.4

Dans le cadre de cet APP, nous considérons les caméras comme étant idéales, c'est-à-dire qu'on peut obtenir leurs caractéristiques intrinsèques et extrinsèques à partir de quelques paramètres seulement.

**a)** Qu'est-ce qui rend les vraies caméras non-idéales ? Nommez des facteurs autant pour les caractéristiques intrinsèques que extrinsèques.

Leur capacité d'ajustements à la quantité de lumière reçue n'est pas parfaites, du bruit ainsi que des aberration dû à l'imperfection des lentilles sphérique (intrinsèque). 

Les surfaces ayant une réflexion diffuse ne sont pas bien capter par l'"Apperture" et l'illumination de l'environnement trop grand ou trop petit peut provoquer des problème et aussi le postionnement de la caméra (extrinsèque)
Réponse : Réalité des imperfections provenant de la fabrication de la caméra

**b)** Que doit on faire pour obtenir les caractéristiques d'une caméra non-idéale ?

En faisant une estimation basé sur plusieurs mesure. On pourrait aussi introduire un pourcentage d'erreur. 
Réponse : Faire une calibration

### Q1.5

Dans cette image synthétique : 

![](https://upload.wikimedia.org/wikipedia/commons/c/cd/Specular_highlight.jpg)

(source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Specular_highlight.jpg))

**a)** Quelle(s) partie(s) correspondent à l'illumination diffuse et les reflets spéculaires ?

Les partie où il y a un passage d'une zone illuminée à plus sombre = illumination diffuse
Réponse : Tout ce qui fait qu'il y a de la couleur

Les partie où il y a des points blancs = reflet spéculaire

**b)** Quelle information est nécessaire pour déterminer les caractéristiques et emplacements exacts des sources de lumières dans cette image ? Vous pouvez répondre en utilisant des éléments de la *Bidirectional Reflectance Distribution Function* (BRDF).

Nous avons besoin de l'angle d'incidence et de réflexion, la longeur d'onde et les angles de rotation par rapport à la normale de la surface pour le rayon incident et réfléchis.
Réponse : C'est très difficile à faire seulement à partir de la photo, on pourrait le faire si on avait beaucoup de points de vue différent

### Q1.6

**a)** Pourquoi deux appareils de capture peuvent produire des valeurs RGB différentes d'une même couleur ? 

Car rien n'empêche les fabricant de caméra de changer un peu les valeurs par la suite pour que l'image correspondent mieux à leur besoin.
Réponse : Les caméras ne sont pas tous calibré de la même façon

**b)** Que peut-on faire pour comparer numériquement des couleurs provenant de deux capteurs différents ?

Il faudrait savoir les informations qui apportent un changement au calcul de la matrice RGB, et faire l'équation 2.104 pour avoir la matrice XYZ. Il sera possible de comparer ces deux matrice pour comparer des bananes avec des bananes.
Réponse : on a besoin de la matrice de conversion

## Repères et coordonnées

### Q2.1

Supposons ces 2 repères :

![](images_doc/proc1-q2_1-frames.png)

**a)** Trouvez la matrice homogène permettant de transformer un point du repère $\{1\}$ au repère $\{0\}$.

In [35]:
T_10 = np.identity(4) # Génère une matrice identité 4x4
t_mat = np.array([-240,-80,120])
t_mat_t = t_mat.transpose()

T_mat = np.array([[0,1,0,0],[1,0,0,0],[0,0,-1,0],[0.24,0.08,0.12,1]])
#On vérifie quel axes sont alligné avec lesquelles, et on met les signes en conséquence
# On pars du repère 0 et on regarder les dimensions, on doit garder des unité
T_mat = T_mat.transpose()

print("T_mat:\n", T_mat)
print("T_10:\n", T_10)

T_mat:
 [[ 0.    1.    0.    0.24]
 [ 1.    0.    0.    0.08]
 [ 0.    0.   -1.    0.12]
 [ 0.    0.    0.    1.  ]]
T_10:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


**b)** Trouvez maintenant la transformation inverse.

In [36]:
T_01 = np.identity(4)
T_mat_t = np.linalg.inv(T_mat)
print(T_mat_t)

[[ 0.    1.    0.   -0.08]
 [ 1.    0.    0.   -0.24]
 [-0.   -0.   -1.    0.12]
 [ 0.    0.    0.    1.  ]]


**c)** Soit le point $p_0 = [8, 5, 1]^T$, un point dans le repère $\{0\}$. Trouvez $p_1$, ses coordonnées dans le repère $\{1\}$.

In [37]:
p_0 = [8, 5, 1, 1]
p_1 = T_mat_t @ p_0
print(p_1)

[ 4.92  7.76 -0.88  1.  ]


### Q2.2

Supposons maintenant que le repère $\{1\}$ représente une caméra avec les caractéristiques intrinsèques $K$ de la question Q1.3.

**a)** Trouvez la matrice de projection P complète permettant de projeter un point $p$ décrit dans le repère $\{0\}$.

In [49]:
# print(K) # Si vous n'avez pas réutilisé la variable K, elle aura toujours la même valeur qu'à la question Q1.3.
K1 = np.zeros((4,4))

K1[0:3,0:3] = K
K1[3,3] = 1
print("K1:\n", K1)

#pourquoi est-ce qu'on doit prendre la matrice T inverse?
P1 = K1 @ T_mat_t
print("P1:\n", P1)



K1:
 [[6.200e+02 0.000e+00 1.024e+03 0.000e+00]
 [0.000e+00 6.200e+02 5.120e+02 0.000e+00]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00]]
P1:
 [[ 0.000e+00  6.200e+02 -1.024e+03  7.328e+01]
 [ 6.200e+02  0.000e+00 -5.120e+02 -8.736e+01]
 [ 0.000e+00  0.000e+00 -1.000e+00  1.200e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


**b)** Soit le point $p_0 = [0.250, 0.010, 0.000]$. Trouvez le point $x_s$, les coordonnées du point $p_0$ perçu par la caméra.

In [52]:
p_0 = np.array([0.250, 0.010, 0.000])

#C'est quoi que ca veut dire d'augmenter une matrice?
p_w = np.array([0.250, 0.010, 0.000,1])

x_s_p = P1@p_w
# print(x_s_p)

x_s = x_s_p/x_s_p[2]
print(x_s)

[662.33333333 563.66666667   1.           8.33333333]


## Reprojection 2D à 3D

### Q3.1

Supposons que le plan XY du repère $\{0\}$ est un convoyeur. Quelle serait sa largeur maximale (mesurée sur l'axe Y) si on souhaite que la caméra la capte au complet dans son image ? 

In [8]:
l_conv = 0

### Q3.2

Soit le point $x_s = [120, 200]$, un point dans l'image perçu par la caméra décrite plus haut. On suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Trouvez les coordonnées du point $p_0$ qui correspond à ce même point dans le repère $\{0\}$.

In [54]:
x_s_2 = np.array([120,200])
P1_inv = np.linalg.inv(P1)
x_rp = x_s/x_s[3]
print(x_rp)



[79.48 67.64  0.12  1.  ]
